## 0. Instalación de librerías externas

In [1]:
!pip install transformers
!pip install sentencepiece
!pip install accelerate
!pip install datasets
!pip install evaluate
!pip install rouge_score

     ---------------------------------------- 0.0/43.8 kB ? eta -:--:--
     ---------------------------------------- 43.8/43.8 kB 2.2 MB/s eta 0:00:00
  Using cached PyYAML-6.0.1-cp310-cp310-win_amd64.whl.metadata (2.1 kB)
  Using cached regex-2024.5.15-cp310-cp310-win_amd64.whl.metadata (41 kB)
   ---------------------------------------- 0.0/9.1 MB ? eta -:--:--
    --------------------------------------- 0.2/9.1 MB 6.3 MB/s eta 0:00:02
   --- ------------------------------------ 0.9/9.1 MB 11.4 MB/s eta 0:00:01
   --------- ------------------------------ 2.1/9.1 MB 17.0 MB/s eta 0:00:01
   ---------------- ----------------------- 3.7/9.1 MB 21.2 MB/s eta 0:00:01
   ------------------------- -------------- 5.8/9.1 MB 26.2 MB/s eta 0:00:01
   -------------------------------- ------- 7.3/9.1 MB 29.1 MB/s eta 0:00:01
   -------------------------------- ------- 7.3/9.1 MB 29.1 MB/s eta 0:00:01
   -------------------------------- ------- 7.3/9.1 MB 20.4 MB/s eta 0:00:01
   ---------------

## 1. Comportamiento de [Flan-T5-small](https://huggingface.co/google/flan-t5-small) sin Fine-tuning

### Lectura del modelo y tokenizador

In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

# Verificar si MPS (Metal Performance Shaders) está disponible y usarlo si es posible
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

# Importar el tokenizador
tokenizer_FT5 = T5Tokenizer.from_pretrained("google/flan-t5-small")

# Importar el modelo pre-entrenado
model_FT5 = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small").to(device)

c:\Users\SantiNR\mplace_meli\venv_meli\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Generación de texto

## 2. Selección y preparación del conjunto de datos

### 2.1 Carga de los Conjuntos de Datos

In [71]:
from datasets import load_dataset, DatasetDict

# Cargar los datos
good_data = load_dataset('csv', data_files='../data/dataset_llm.csv')

good_df = good_df[['title', 'price', 'CATEGORIA_CODE']]
good_df = good_df[good_df['CATEGORIA_CODE']=='MLA1000'].head(50)


# # Cargar los datos malos
# bad_data = load_dataset('csv', data_files='/Users/feliperangel/LLMs/mvp_0_Instruction_Fine_tuning_LLM_Base/data/ping_bad.csv')

# Unir los conjuntos de datos en un DatasetDict
datasets = DatasetDict({
    'marketplace': good_data['train']
})

print(datasets)

DatasetDict({
    marketplace: Dataset({
        features: ['title', 'price', 'original_price', 'CATEGORIA_CODE'],
        num_rows: 912102
    })
})


## 2.2 Visualización y Exploración de los Datos

In [72]:
# Mostrar una muestra de los datos buenos
print(datasets['marketplace'].to_pandas().head())

                                               title    price  original_price  \
0  Auriculares Bluetooth In-ear Gamer F9-5 Negro ...   7599.0          9999.0   
1      Auriculares Inalámbricos Jbl Tune 520bt Negro  68999.0             NaN   
2  Auriculares In-ear Inalámbricos Xiaomi Redmi B...  33894.0             NaN   
3  Parlante Philco Djp11p Portátil Con Bluetooth ...  59999.0         78999.0   
4  Pila Aa Energizer Max E91 Cilíndrica - Pack De...   4200.0             NaN   

  CATEGORIA_CODE  
0        MLA1000  
1        MLA1000  
2        MLA1000  
3        MLA1000  
4        MLA1000  


## 2.3 Preprocesamiento y Manipulación de Datos

In [75]:
import pandas as pd
from datasets import Dataset, DatasetDict, concatenate_datasets

# Cargar los datos buenos
good_df = pd.read_csv('../data/dataset_llm.csv', usecols=['title', 'price', 'CATEGORIA_CODE'])

good_df = good_df[good_df['CATEGORIA_CODE']=='MLA1000'].head(50)

# Asegurarse de que fltvalue sea de tipo float64
good_df['price'] = good_df['price'].astype('float64')

# Convertir el DataFrame a Dataset
good_data = Dataset.from_pandas(good_df)

# # Cargar los datos malos
# bad_df = pd.read_csv('../data/dataset_llm.csv')
# # Asegurarse de que fltvalue sea de tipo float64
# bad_df['fltvalue'] = bad_df['fltvalue'].astype('float64')

# # Convertir el DataFrame a Dataset
# bad_data = Dataset.from_pandas(bad_df)

# Crear un DatasetDict
datasets = DatasetDict({
    'marketplace': good_data,
})

# Función para etiquetar los datos
def label_data(example, label):
    example['label'] = label
    return example

# Etiquetar los datos buenos con '1' y los malos con '0'
datasets['marketplace'] = datasets['marketplace'].map(lambda x: label_data(x, 1), batched=False)
#datasets['bad'] = datasets['bad'].map(lambda x: label_data(x, 0), batched=False)

# Unir los conjuntos etiquetados en un solo conjunto de datos
all_data = concatenate_datasets([datasets['marketplace']
                                 #, datasets['bad']
                                 ])

# Verificar la estructura y algunas muestras de los datos
print(all_data.column_names)  # Debería incluir la nueva columna 'label'
print(all_data)

# Mostrar una muestra de los datos buenos y malos
print(all_data.to_pandas().head())

Map: 100%|██████████| 50/50 [00:00<00:00, 6242.08 examples/s]

['title', 'price', 'CATEGORIA_CODE', '__index_level_0__', 'label']
Dataset({
    features: ['title', 'price', 'CATEGORIA_CODE', '__index_level_0__', 'label'],
    num_rows: 50
})
                                               title    price CATEGORIA_CODE  \
0  Auriculares Bluetooth In-ear Gamer F9-5 Negro ...   7599.0        MLA1000   
1      Auriculares Inalámbricos Jbl Tune 520bt Negro  68999.0        MLA1000   
2  Auriculares In-ear Inalámbricos Xiaomi Redmi B...  33894.0        MLA1000   
3  Parlante Philco Djp11p Portátil Con Bluetooth ...  59999.0        MLA1000   
4  Pila Aa Energizer Max E91 Cilíndrica - Pack De...   4200.0        MLA1000   

   __index_level_0__  label  
0                  0      1  
1                  1      1  
2                  2      1  
3                  3      1  
4                  4      1  


## 2.4 División en Entrenamiento y Prueba 

In [76]:
# Dividir los datos buenos y malos en entrenamiento y prueba
train_test_split_good = datasets['marketplace'].train_test_split(test_size=0.2)
# train_test_split_bad = datasets['bad'].train_test_split(test_size=0.2)

# Concatenar los conjuntos de entrenamiento y prueba
# train_data = concatenate_datasets([train_test_split_good['train'], train_test_split_bad['train']])
# test_data = concatenate_datasets([train_test_split_good['test'], train_test_split_bad['test']])
train_data = train_test_split_good['train']
test_data = train_test_split_good['test']

# Crear un nuevo DatasetDict con las divisiones
split_datasets = DatasetDict({
    'train': train_data,
    'test': test_data
})

print(split_datasets)

DatasetDict({
    train: Dataset({
        features: ['title', 'price', 'CATEGORIA_CODE', '__index_level_0__', 'label'],
        num_rows: 40
    })
    test: Dataset({
        features: ['title', 'price', 'CATEGORIA_CODE', '__index_level_0__', 'label'],
        num_rows: 10
    })
})


### 3 Formato del conjunto de datos

In [101]:
import pandas as pd
from datasets import Dataset, DatasetDict, concatenate_datasets

# Cargar los datos buenos
good_df = pd.read_csv('../data/dataset_llm.csv', usecols=['title', 'price', 'CATEGORIA_CODE'])
good_df = good_df[good_df['CATEGORIA_CODE']=='MLA1000'].head(50)
good_df['price'] = good_df['price'].astype('float64')
good_df['label'] = 1

# # Cargar los datos malos
# bad_df = pd.read_csv('/Users/feliperangel/LLMs/mvp_0_Instruction_Fine_tuning_LLM_Base/data/ping_bad.csv')
# bad_df['fltvalue'] = bad_df['fltvalue'].astype('float64')
# bad_df['label'] = 0

# Concatenar los datos
df = good_df#pd.concat([good_df, bad_df])

# Crear el prompt y el completion
def create_prompt_completion(row):
    # Formato el precio como float con dos decimales para mayor claridad
    price_formatted = f"{row['price']:.2f} pesos argentinos"

    prompt = (
        f"Memorize the relationship between the 'title' and 'price' data of items in a Marketplace\n\n"
        f"title: {row['title']}\n"
        f"price: {price_formatted}\n"  # Incluyo el precio formateado con dos decimales y la unidad
        f"CATEGORIA_CODE: {row['CATEGORIA_CODE']}\n"
        f"<sep>\n"
    )

    # Completo la categoría basada en el código de categoría
    completion = "Electrónica y audio" if row['CATEGORIA_CODE'] == "MLA1000" else "Otra Categoría"

    return {"prompt": prompt, "completion": completion}
    # prompt = (
    #     f"Memoriza la relación entre la data de 'title' y 'price' de los artículos de un Marketplace \n\n"
    #     f"title: {row['title']}\n"
    #     f"price: {row['price']}\n"
    #     f"CATEGORIA_CODE: {row['CATEGORIA_CODE']}\n"
    #     f"<sep>\n"
    # )
    # completion = "Electrónica y audio" if row['CATEGORIA_CODE'] == "MLA1000" else "Otra Categoría"
    # return {"prompt": prompt, "completion": completion}

# Aplicar la función a cada fila
formatted_data = df.apply(create_prompt_completion, axis=1)

# Convertir a Dataset de Hugging Face
dataset = Dataset.from_pandas(pd.DataFrame(formatted_data.tolist()))

# Dividir en entrenamiento y prueba
train_test_split = dataset.train_test_split(test_size=0.2)
split_datasets = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test']
})

# Reducir el tamaño del conjunto de datos
NUM_EJ_TRAIN = 1500
NUM_EJ_VAL = 500
NUM_EJ_TEST = 200

# Subconjunto de entrenamiento
split_datasets['train'] = split_datasets['train'].select(range(min(NUM_EJ_TRAIN, len(split_datasets['train']))))

# Subconjunto de validación (utilizando parte del conjunto de prueba)
split_datasets['validation'] = split_datasets['test'].select(range(min(NUM_EJ_VAL, len(split_datasets['test']))))

# Subconjunto de pruebas
split_datasets['test'] = split_datasets['test'].select(range(min(NUM_EJ_TEST, len(split_datasets['test']))))

print(split_datasets)

# Mostrar una muestra de los datos buenos y malos
print(split_datasets['train'].to_pandas().head())

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 40
    })
    test: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 10
    })
    validation: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 10
    })
})
                                              prompt           completion
0  Memorize the relationship between the 'title' ...  Electrónica y audio
1  Memorize the relationship between the 'title' ...  Electrónica y audio
2  Memorize the relationship between the 'title' ...  Electrónica y audio
3  Memorize the relationship between the 'title' ...  Electrónica y audio
4  Memorize the relationship between the 'title' ...  Electrónica y audio


### 4. Tokenización del conjunto de datos

In [102]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Tokenización y padding
tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-small')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Una de las cosas que comentabamos cuándo comenzamos a hablar de modelos generativos como la Redes Neuronales Recurrentes, es que este tipo de algoritmos, al igual que los LLMs, reciben secuencias del mismo tamaño.

Con lo cual, al igual que hicimos en ese caso práctico al comienzo del curso, debemos obtener la secuencia más larga de nuestro conjunto de datos y realizar padding al resto de secuencias para que todas tengan el mismo tamaño.

In [103]:
from datasets import concatenate_datasets

# Calculamos el tamaño máximo de prompt
prompts_tokens = concatenate_datasets([split_datasets["train"], split_datasets["validation"], split_datasets["test"]]).map(lambda x: tokenizer(x["prompt"], truncation=True), batched=True)
max_prompt_len = max([len(x) for x in prompts_tokens["input_ids"]])
print(f"Maximo tamaño de prompt: {max_prompt_len}")

# Calculamos el tamaño máximo de completion
completions_tokens = concatenate_datasets([split_datasets["train"], split_datasets["validation"], split_datasets["test"]]).map(lambda x: tokenizer(x["completion"], truncation=True), batched=True)
max_completion_len = max([len(x) for x in completions_tokens["input_ids"]])
print(f"Maximo tamaño de completion: {max_completion_len}")


Map: 100%|██████████| 60/60 [00:00<00:00, 1318.19 examples/s]


Maximo tamaño de prompt: 103


Map: 100%|██████████| 60/60 [00:00<00:00, 6000.29 examples/s]

Maximo tamaño de completion: 9


In [104]:
# Función de tokenización y padding
def padding_tokenizer(datos):
    # Tokenizar inputs (prompts)
    model_inputs = tokenizer(datos['prompt'], max_length=max_prompt_len, padding="max_length", truncation=True)

    # Tokenizar labels (completions)
    model_labels = tokenizer(datos['completion'], max_length=max_completion_len, padding="max_length", truncation=True)

    # Sustituimos el caracter de padding de las completion por -100 para que no se tenga en cuenta en el entrenamiento
    model_labels["input_ids"] = [[(l if l != tokenizer.pad_token_id else -100) for l in label] for label in model_labels["input_ids"]]

    model_inputs['labels'] = model_labels["input_ids"]

    return model_inputs

# Aplicar tokenización y padding a los conjuntos de datos
split_datasets = split_datasets.map(padding_tokenizer, batched=True, remove_columns=['prompt', 'completion'])

# Verificar la estructura y algunas muestras de los datos tokenizados
print(split_datasets)

# Mostrar una muestra de los datos tokenizados
print(split_datasets['train'].to_pandas().head())

Map: 100%|██████████| 10/10 [00:00<00:00, 1251.06 examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 40
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10
    })
})
                                           input_ids  \
0  [23018, 52, 1737, 8, 1675, 344, 8, 3, 31, 2186...   
1  [23018, 52, 1737, 8, 1675, 344, 8, 3, 31, 2186...   
2  [23018, 52, 1737, 8, 1675, 344, 8, 3, 31, 2186...   
3  [23018, 52, 1737, 8, 1675, 344, 8, 3, 31, 2186...   
4  [23018, 52, 1737, 8, 1675, 344, 8, 3, 31, 2186...   

                                      attention_mask  \
0  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
1  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
2  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
3  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
4  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1

## 3. Fine-tuning del modelo

### 3.1. Lectura del modelo

In [105]:
from transformers import AutoModelForSeq2SeqLM

# Cargamos el modelo
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

### 5.2. Evaluación durante el entrenamiento

A continuación implementamos un conjunto de funciones auxiliares para evluar los resultados durante el proceso de entrenamiento

In [106]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metrica de evaluación
metric = evaluate.load("rouge")

# Funciona auxiliar para preprocesar el texto
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum espera una nueva línea después de cada frase
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Reemplazamos -100 en las etiquetas porque no podemos decodificarlo
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Preprocesamos el texto
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\SantiNR\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 5.3. Lectura y adaptación de los datos para el entrenamiento

In [107]:
from transformers import DataCollatorForSeq2Seq

# Ignoramos los tokens relacionados con el padding durante el proceso de entrenamiento para los prompts
label_pad_token_id = -100

# Recolector de datos para el entrenamiento del modelo
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)


### 5.3.1 Preparación y ejecución del fine-tuning (entrenamiento)

In [108]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

REPOSITORY="flan-t5-small-fine-tuned-test50"

# Definimos las opciones del entrenamiento
# Definimos las opciones del entrenamiento
training_args = Seq2SeqTrainingArguments(
    # Hiperprámetros del entrenamiento
    output_dir=REPOSITORY,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False,  # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=4,
    # Estrategias de logging y evaluación
    logging_dir=f"{REPOSITORY}/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
)

# Creamos la instancia de entrenamiento
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=split_datasets["train"],  # Usar split_datasets en lugar de ds_tokens
    eval_dataset=split_datasets["validation"],  # Usar split_datasets en lugar de ds_tokens
    compute_metrics=compute_metrics,
)

c:\Users\SantiNR\mplace_meli\venv_meli\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [109]:
# Iniciar el entrenamiento del modelo
trainer.train()

  0%|          | 0/20 [00:00<?, ?it/s]

 25%|██▌       | 5/20 [00:13<00:34,  2.31s/it]c:\Users\SantiNR\mplace_meli\venv_meli\lib\site-packages\transformers\generation\utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(



                                             

                                              
 25%|██▌       | 5/20 [00:16<00:34,  2.31s/it]


{'eval_loss': 2.5527853965759277, 'eval_rouge1': 0.0, 'eval_rouge2': 0.0, 'eval_rougeL': 0.0, 'eval_rougeLsum': 0.0, 'eval_gen_len': 18.6, 'eval_runtime': 3.4736, 'eval_samples_per_second': 2.879, 'eval_steps_per_second': 0.576, 'epoch': 1.0}


 50%|█████     | 10/20 [00:26<00:21,  2.17s/it]c:\Users\SantiNR\mplace_meli\venv_meli\lib\site-packages\transformers\generation\utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(



                                             

                                               
 50%|█████     | 10/20 [00:29<00:21,  2.17s/it]


{'eval_loss': 1.8520581722259521, 'eval_rouge1': 0.0, 'eval_rouge2': 0.0, 'eval_rougeL': 0.0, 'eval_rougeLsum': 0.0, 'eval_gen_len': 16.8, 'eval_runtime': 2.8338, 'eval_samples_per_second': 3.529, 'eval_steps_per_second': 0.706, 'epoch': 2.0}


...

100%|██████████| 20/20 [00:52<00:00,  2.09s/it]c:\Users\SantiNR\mplace_meli\venv_meli\lib\site-packages\transformers\generation\utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(



                                             

                                               
100%|██████████| 20/20 [00:54<00:00,  2.09s/it]


{'eval_loss': 1.3082988262176514, 'eval_rouge1': 1.8182, 'eval_rouge2': 0.0, 'eval_rougeL': 1.8182, 'eval_rougeLsum': 1.8182, 'eval_gen_len': 17.4, 'eval_runtime': 2.7258, 'eval_samples_per_second': 3.669, 'eval_steps_per_second': 0.734, 'epoch': 4.0}


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].

100%|██████████| 20/20 [00:56<00:00,  2.81s/it] 

{'train_runtime': 56.1678, 'train_samples_per_second': 2.849, 'train_steps_per_second': 0.356, 'train_loss': 2.7374202728271486, 'epoch': 4.0}


TrainOutput(global_step=20, training_loss=2.7374202728271486, metrics={'train_runtime': 56.1678, 'train_samples_per_second': 2.849, 'train_steps_per_second': 0.356, 'total_flos': 6041441402880.0, 'train_loss': 2.7374202728271486, 'epoch': 4.0})

In [110]:
trainer.save_model('../model/'+REPOSITORY)
tokenizer.save_pretrained('../model/'+REPOSITORY)

('../model/flan-t5-small-fine-tuned-test50\\tokenizer_config.json',
 '../model/flan-t5-small-fine-tuned-test50\\special_tokens_map.json',
 '../model/flan-t5-small-fine-tuned-test50\\spiece.model',
 '../model/flan-t5-small-fine-tuned-test50\\added_tokens.json')

In [111]:
# Evaluar el modelo en el conjunto de validación
eval_results = trainer.evaluate()
print(f"Resultados de la evaluación: {eval_results}")


c:\Users\SantiNR\mplace_meli\venv_meli\lib\site-packages\transformers\generation\utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 2/2 [00:01<00:00,  1.93it/s]

Resultados de la evaluación: {'eval_loss': 1.3082988262176514, 'eval_rouge1': 1.8182, 'eval_rouge2': 0.0, 'eval_rougeL': 1.8182, 'eval_rougeLsum': 1.8182, 'eval_gen_len': 17.4, 'eval_runtime': 3.6959, 'eval_samples_per_second': 2.706, 'eval_steps_per_second': 0.541, 'epoch': 4.0}


In [112]:
# Define a custom prompt for testing
test_prompt = "Cuál es el price del iphone"

# Tokenize the input prompt
inputs = tokenizer(test_prompt, return_tensors="pt").to(device)

# Generate a response
output_sequences = model.generate(
    input_ids=inputs['input_ids'],
    max_length=max_completion_len,  # You can set this to an appropriate length
    num_return_sequences=1,  # Number of generated sequences
    no_repeat_ngram_size=2,  # Prevent repetition
    early_stopping=True
)

# Decode the generated sequence
generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
print("Generated Text:", generated_text)


c:\Users\SantiNR\mplace_meli\venv_meli\lib\site-packages\transformers\generation\configuration_utils.py:563: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Generated Text: $9.99


In [59]:
from transformers import T5Tokenizer, AutoModelForSeq2SeqLM
import torch

# Verificar si MPS (Metal Performance Shaders) está disponible y usarlo si es posible
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

# Tokenización y padding
tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-small')

# Cargamos el modelo
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small").to(device)

# Función para generar texto con una longitud específica
def generate_text(prompt, max_length=18):
    # Tokenizar el prompt de entrada
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Generar una secuencia de salida con la longitud máxima deseada
    output_sequences = model.generate(
        input_ids=inputs['input_ids'],
        max_length=max_length,  # Aquí especificamos la longitud máxima deseada
        num_return_sequences=1,  # Número de secuencias generadas
        no_repeat_ngram_size=2,  # Evitar repeticiones
        early_stopping=True
    )

    # Decodificar la secuencia generada
    generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)

    return generated_text

# Ejemplo de uso
#test_prompt = "Cuál es el price del iphone?"
#test_prompt = "Cuál es el price del title parlante?"
test_prompt = "What is the price of the product with title 'Auriculares' in Argentine pesos?"


generated_text = generate_text(test_prompt, max_length=12)
print("Generated Text (12 characters):", generated_text)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Generated Text (12 characters): £20


In [125]:
from transformers import T5Tokenizer, AutoModelForSeq2SeqLM
import torch

# Verificar si MPS (Metal Performance Shaders) está disponible y usarlo si es posible
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

# Tokenización y padding
tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-small')

# Cargamos el modelo
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small").to(device)

# Función para generar texto con una longitud específica
def generate_text(prompt, max_length=50):
    # Tokenizar el prompt de entrada
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Generar una secuencia de salida con la longitud máxima deseada
    output_sequences = model.generate(
        input_ids=inputs['input_ids'],
        max_length=max_length,  # Aquí especificamos la longitud máxima deseada
        num_return_sequences=1,  # Número de secuencias generadas
        no_repeat_ngram_size=2,  # Evitar repeticiones
        early_stopping=True
    )

    # Decodificar la secuencia generada
    generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)

    return generated_text

# Ejemplo de uso con un prompt más específico
test_prompt = "Price of 'Auriculares In-ear Inalámbricos'?"
#test_prompt = "What is the 'CATEGORIA_CODE' of the product 'Parlante'?"


generated_text = generate_text(test_prompt, max_length=50)
print("Generated Text:", generated_text)

# Implementar lógica para extraer la información relevante del texto generado
def parse_generated_text(generated_text):
    # Regular expression pattern to find numbers (including decimals)
    number_pattern = r'(\d+(\.\d+)?)'

    # Search for the number in the generated text
    match = re.search(number_pattern, generated_text)
    if match:
        extracted_price = match.group(0)  # Get the matched number as a string
        return extracted_price
    else:
        return "Price not found"  # Handle case where price is not found

# Ejemplo de cómo podrías usar la función parse_generated_text
price_answer = parse_generated_text(generated_text)
print("Extracted Price:", price_answer)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Generated Text: £30
Extracted Price: 30


In [114]:
good_df

,title,price,CATEGORIA_CODE,label
0,Auriculares Bluetooth In-ear Gamer F9-5 Negro ...,7599.00,MLA1000,1
1,Auriculares Inalámbricos Jbl Tune 520bt Negro,68999.00,MLA1000,1
2,Auriculares In-ear Inalámbricos Xiaomi Redmi B...,33894.00,MLA1000,1
3,Parlante Philco Djp11p Portátil Con Bluetooth ...,59999.00,MLA1000,1
4,Pila Aa Energizer Max E91 Cilíndrica - Pack De...,4200.00,MLA1000,1
5,Google Chromecast 4 Generación Con Google Tv H...,122795.00,MLA1000,1
6,Chromecast Google Ga03131 Tv Hd 8gb 2gb Ram 60...,101000.00,MLA1000,1
7,Auriculares In-ear Gamer Inalámbricos Tws F9-5...,7599.00,MLA1000,1
8,20 X Pilas Aaa Energizer Max E92 Pilas Alcalin...,12700.00,MLA1000,1
9,Televisor Tcl Led 32s5400af Android Tv 32 Full...,295999.00,MLA1000,1
